## 증권 뉴스 크롤링

In [1]:
import requests
import pandas as pd
import pandas as DataFrame
from bs4 import BeautifulSoup
import time

def get_news(date, pages):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36'
    }
    url = "https://finance.naver.com/news/news_list.nhn?mode=RANK&date=" + str(date) + "&page=" + str(pages)
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    lists = []
    #hot_news= soup.find_all(class_ = 'hotNewsList')
    hot_news = soup.select('.hotNewsList .newsList li li')
    time.sleep(3)
    newsNum = 0
    for item in hot_news:
        d = {}
        #d['LinkSrc'] = item.find('a')['href']
        newsNum += 1
        d['NewsNo'] = newsNum
        d['Title'] = item.find('a')['title']
        d['Press'] = item.find(class_="press").get_text()
        lists.append(d)
    return lists
#a = get_news(20200921,1)
#print(type(a))
#print(a)

In [2]:
get_news(20200925,1)

[{'NewsNo': 1, 'Title': '수젠텍, 손미진 대표 외 임원진 자사주 32만주 취득', 'Press': '매일경제 '},
 {'NewsNo': 2, 'Title': '요즘 증시…이 종목 떨어지면 어김없이 코스피도 빠졌다', 'Press': '매일경제 '},
 {'NewsNo': 3, 'Title': '[fn오전시황] 코스피, 기관 매수에 상승 반전', 'Press': '파이낸셜뉴스 '},
 {'NewsNo': 4, 'Title': '"연말 배당 노려볼까" 기술주 조정 국면서 재부상하는 배당주', 'Press': '데일리안 '},
 {'NewsNo': 5, 'Title': '현대차증권, ‘이스트스프링 美회사채 펀드’ 추천', 'Press': '데일리안 '},
 {'NewsNo': 6, 'Title': '[유럽증시] 코로나19 재확산·제한조치 도입에 일제히 하락', 'Press': '연합뉴스 '},
 {'NewsNo': 7,
  'Title': '뉴욕증시, 엇갈린 美 지표 속 변동성 장세…다우, 0.2% 상승 마감',
  'Press': '연합뉴스 '},
 {'NewsNo': 8, 'Title': '현대차 노조, 임금 동결 잠정합의안 찬반투표 시작', 'Press': '연합뉴스 '},
 {'NewsNo': 9, 'Title': '코스피 반등 출발…장중 2,290대로 상승', 'Press': '연합뉴스 '},
 {'NewsNo': 10, 'Title': '코스피 반등 출발', 'Press': '연합뉴스 '},
 {'NewsNo': 11, 'Title': '코스피, 2,290대로 상승', 'Press': '연합뉴스 '},
 {'NewsNo': 12,
  'Title': '코스피 기관 매수에 반등 출발…장중 2,290대 상승(종합)',
  'Press': '연합뉴스 '},
 {'NewsNo': 13, 'Title': '올해 임협 잠정합의안 찬반투표하는 현대차 조합원', 'Press': '연합뉴스 '},
 {'NewsNo': 14, 'Title': '금융결

In [3]:
#pip install flask_cors

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, jsonify, request
import json
from flask_cors import CORS, cross_origin
from flask_restful import Resource, Api, reqparse
app = Flask(__name__)
api = Api(app)
CORS(app, resources={r'*': {'origins': 'http://localhost:8080'}})
@app.route('/newsList',methods=['GET'])
def new_data():
    print("It's operate!!")
    formatDate = request.args.get('formatDate')
    print("formatDate : " + formatDate)
    data_list = get_news(formatDate,1)
    return jsonify(data_list)
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


It's operate!!
formatDate : 20201015


127.0.0.1 - - [15/Oct/2020 11:32:56] "GET /newsList?formatDate=20201015 HTTP/1.1" 200 -


## 시가총액 크롤링

In [28]:
MARKET_URL='https://finance.naver.com/sise/sise_market_sum.nhn?sosok='
KOSPI_CODE = 0
KOSDAK_CODE = 1
fields = []
START_PAGE = 1
def mainPage(code):
    res = requests.get(MARKET_URL + str(code) + "&page=" + str(START_PAGE))
    page_soup = BeautifulSoup(res.text, 'lxml')
    top10_page_num = page_soup.select_one('td.pgRR > a')
    top10_page_num = int(top10_page_num.get('href').split('=')[-1])
    # 항목명들 추출
    ipt_html = page_soup.select_one('div.subcnt_sise_item_top')
    global fields
    fields = [item.get('value') for item in ipt_html.select('input')]
    result = [crawl(code, str(page)) for page in range(1, top10_page_num+1)]
    df = pd.concat(result, axis=0, ignore_index=True)
    
def crawl(code, page):
    global fields
    data = {'menu': 'market_sum', 'fieldIds': fields,
            'returnUrl': MARKET_URL + str(code) + "&page=" + str(page)}
    #Post 요청 (requests.get 요청 대신)
    res = requests.post('https://finance.naver.com/sise/field_submit.nhn', data=data)
    page_soup = BeautifulSoup(res.text, 'lxml')
    table_html = page_soup.select_one('div.box_type_l')
    # Column 이름
    col_name_data = [item.get_text().strip() for item in table_html.select('thead th')][1:-1]
    # 종목명 + 수치 추출 (a.title = 종목명, td.numb = 기타 수치)
    inner_data = [item.get_text().strip() for item in table_html.find_all(lambda x:
                                                                         (x.name == 'a' and
                                                                         'tltle' in x.get('class', [])) or
                                                                         (x.name == 'td' and
                                                                         'number' in x.get('class', [])))]
    # 종목의 순번
    no_data = [item.get_text().strip() for item in table_html.select('td.no')]
    number_data = np.array(inner_data)
    
    for item in range(10):
        dict = {}
        dict['no'] = no_data
        dict['company'] = number_data[0]
        dict['current'] = number_data[1]
        dict['pre'] = number_data[2]
        dict['updown'] = number_data[3]
        dict['face'] = number_data[4]
        dict['volume'] = number_data[5]
        dict['preVolume'] = number_data[7]
        dict['price'] = number_data[8]
        dict
    
    print(number_data)
    # 행렬화 (크기에 맞게)
    #number_data.resize(len(no_data), len(col_name_data))
    #df = pd.DataFrame(data=number_data, columns =  col_name_data)
    #return df

In [29]:
mainPage(0)

['삼성전자' '58,000' '200' ... '7.56' '4.91' '3,174.7']
['LG유플러스' '10,850' '150' ... '4.14' '1.80' '138.8']
['한국가스공사' '23,900' '200' ... '0.04' '0.08' '243.5']
['KCC' '142,500' '0' ... '3.40' '0.45' '2,869.1']
['제이알글로벌리츠' '4,900' '10' ... '4.11' '1.26' '234.6']
['동국제강' '6,220' '140' ... '7.67' '0.46' '260.2']
['스카이라이프' '9,120' '50' ... '-0.03' '1.56' '1,289.7']
['KINDEX 단기통안채' '100,805' '0' ... 'N/A' '0.24' 'N/A']
['세아홀딩스' '69,600' '100' ... '0.21' '0.51' '3,409.0']
['선진' '9,450' '0' ... '2.77' '0.24' '20,998.0']
['삼성출판사' '18,350' '150' ... '1.69' '0.32' '4,562.3']
['ARIRANG 고배당주' '8,780' '90' ... 'N/A' 'N/A' 'N/A']
['BYC' '216,500' '2,500' ... 'N/A' 'N/A' 'N/A']
['한국ANKOR유전' '1,645' '15' ... 'N/A' '2.85' '256.0']
['강남제비스코' '15,150' '100' ... '1.46' '0.92' '1,220.2']
['KODEX KRX300' '14,235' '120' ... 'N/A' 'N/A' 'N/A']
['대현' '1,710' '5' ... 'N/A' 'N/A' 'N/A']
['한익스프레스' '5,200' '0' ... 'N/A' 'N/A' 'N/A']
['미래산업' '3,070' '45' ... 'N/A' 'N/A' 'N/A']
['KBSTAR 200선물레버리지' '12,805' '150' ... 'N/

ValueError: All objects passed were None